In [4]:
import os.path as osp
import os
import sys
from pathlib import Path

import argparse
from matplotlib import pyplot as plt
import matplotlib
import numpy as np
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import gpflowSlim as gfs
# sys.path.append(osp.dirname(osp.dirname(osp.abspath(__file__))))
sys.path.append(osp.dirname(osp.dirname(Path().resolve())))
from data import SyntheticData
from utils.utils import default_plotting_new as init_plotting
from utils.nets import get_posterior
from core.fvi import EntropyEstimationFVI
from utils.logging import get_logger
from utils.neural_kernel import NeuralSpectralKernel, NeuralGibbsKernel
from utils.utils import median_distance_local

matplotlib.use('Agg')

ModuleNotFoundError: No module named 'data'

In [5]:
Path().resolve()

PosixPath('/tmp/af20451a-6e0a-4d47-9371-9c6f680a2935')

In [ ]:
dataset = SyntheticData(args.dataset, return_param_values=False, train_test_split=0.5)
original_x_train, original_y_train = dataset.train_samples()
mean_x, std_x = np.mean(original_x_train), np.std(original_x_train)
mean_y, std_y = np.mean(original_y_train), np.std(original_y_train)
train_x = (original_x_train - mean_x) / std_x
train_y = (original_y_train - mean_y) / std_y
original_x_test, original_y_test = dataset.test_samples()
test_x = (original_x_test - mean_x) / std_x
test_y = (original_y_test - mean_y) / std_y

# y_logstd = np.log(dataset.y_std / std_y)
y_logstd = np.log(3. / std_y)

lower_ap = np.minimum(np.min(train_x), np.min(test_x))
upper_ap = np.maximum(np.max(train_x), np.max(test_x))